In [1]:
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import load_model
import os


# Loading the dataset
path_to_data = "CNRPark+EXT.csv"
data = pd.read_csv(path_to_data, low_memory=False)

# Taking a smaller subset of the data for testing
data_subset = data.sample(n=500, random_state=42)

# Extracting features (images) and labels
features = []
labels = []

for index, row in data_subset.iterrows():
    image_path = row['image_url']
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Checking if the image is loaded successfully
    if image is not None:
        # Resizing the image for consistency
        image = cv2.resize(image, (100, 100))

        # Checking if the resized image has valid dimensions
        if image.shape == (100, 100):
            features.append(image)
            labels.append(row['occupancy'])
        else:
            print(f"Invalid dimensions after resizing for image: {image_path}")

    else:
        print(f"Failed to load image: {image_path}")

features = np.array(features)
labels = np.array(labels)

# Splitting the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)

# Normalizing the data
x_train = x_train / 255.0
x_test = x_test / 255.0

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Building a larger CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the larger model
model.fit(x_train.reshape(-1, 100, 100, 1), y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluating the larger model on the test set
test_loss, test_acc = model.evaluate(x_test.reshape(-1, 100, 100, 1), y_test)
print("Test Accuracy:", test_acc)

# Choosing a random full-scale image from the dataset
random_index = np.random.randint(0, len(data))
folder = "CNR-EXT_FULL_IMAGE_1000x750/FULL_IMAGE_1000x750"
weather_initial = data.iloc[random_index]['weather']
weather_mapping = {'S': 'SUNNY', 'R': 'RAINY', 'O': 'OVERCAST'}
weather = weather_mapping.get(weather_initial, 'UNKNOWN')

capt_date = f"{data.iloc[random_index]['year']}-{data.iloc[random_index]['month']:02d}-{data.iloc[random_index]['day']:02d}"
cam_id = f"camera{str(int(data.iloc[random_index]['camera']))}"  # Ensure the camera value is treated as a string
capt_time = f"{data.iloc[random_index]['hour']:02d}{data.iloc[random_index]['minute']:02d}"

full_image_path = f"{folder}/{weather}/{capt_date}/{cam_id}/{capt_date}_{capt_time}.jpg"

print("Full Image Path:", full_image_path)


# Reading the corresponding CSV file for the selected cam_id
csv_filename = f"{cam_id}.csv"
csv_path = os.path.join("C:\\Users\\vinay\\Downloads\\Dataset_CV_Final_Project\\CNR-EXT_FULL_IMAGE_1000x750", csv_filename)
patch_data = pd.read_csv(csv_path)

# Loading the image
full_image = cv2.imread(full_image_path)


# Drawing red and green boxes based on patch information
for index, patch_info in patch_data.iterrows():
    slot_id = patch_info['SlotId']
    
    # Rescaling the bounding box coordinates to match the current resolution
    x = int(patch_info['X'] * (1000 / 2592))
    y = int(patch_info['Y'] * (750 / 1944))
    w = int(patch_info['W'] * (1000 / 2592))
    h = int(patch_info['H'] * (750 / 1944))
    
    # Ensuring the rescaled region of interest is within the image bounds
    if y < full_image.shape[0] and x < full_image.shape[1] and y + h < full_image.shape[0] and x + w < full_image.shape[1]:
        # Check if the slot is occupied (1) or vacant (0)
        roi = cv2.resize(full_image[y:y+h, x:x+w], (100, 100))
        roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)  # Explicitly converting to single-channel
        if model.predict(np.expand_dims(roi_rgb, axis=0)) > 0.5:
            color = (0, 0, 255)  # red for occupied
        else:
            color = (0, 255, 0)  # green for vacant
        cv2.rectangle(full_image, (x, y), (x + w, y + h), color, 2)

# Saving the resulting image
output_image_path = "output_image.jpg"
cv2.imwrite(output_image_path, full_image)

# Displaying the path to the saved image
print("Image with Boxes saved at:", output_image_path)






Epoch 1/10


10/10 [==============================] - 4s 235ms/step - loss: 0.7017 - accuracy: 0.5200 - val_loss: 0.6284 - val_accuracy: 0.4800
Epoch 2/10
10/10 [==============================] - 2s 193ms/step - loss: 0.5892 - accuracy: 0.6867 - val_loss: 0.5013 - val_accuracy: 0.8133
Epoch 3/10
10/10 [==============================] - 2s 207ms/step - loss: 0.4532 - accuracy: 0.8300 - val_loss: 0.4018 - val_accuracy: 0.8000
Epoch 4/10
10/10 [==============================] - 2s 191ms/step - loss: 0.3047 - accuracy: 0.9000 - val_loss: 0.2171 - val_accuracy: 0.9333
Epoch 5/10
10/10 [==============================] - 2s 191ms/step - loss: 0.2548 - accuracy: 0.9133 - val_loss: 0.1861 - val_accuracy: 0.9467
Epoch 6/10
10/10 [==============================] - 2s 188ms/step - loss: 0.2123 - accuracy: 0.9333 - val_loss: 0.1770 - val_accuracy: 0.9467
Epoch 7/10
10/10 [==============================] - 2s 189ms/step - loss: 0.1860 - accuracy: 0.9400 - val_loss: 0.7027 - val_accuracy: 0.6933
